**Disclaimer** Dit Jupyter Notebook is gemaakt door mij, Winny de Jong, eind juli 2019. Ik ben ervan uit gegaan dat je de 4e module van de gratis online videocursus '[Python for Journalists](https://datajournalism.com/watch/python-for-journalists)' heb gevolgd - in zekere zin bouwt deze notebook daar op voort. Veel plezier!

In [129]:
# setup: importeer de tools die we nodig gaan hebben
# let op Pandas, BeautifulSoup en Requests moet je eerst installeren in je environment
import pandas as pd
from bs4 import BeautifulSoup
import requests

# onderstaande tools zitten automatisch in Python,
# deze tools activeer je dus wel, maar hoef je niet eerst apart te installeren
import csv
import re
import random

## Aantekeningen
Ik wil een spreadsheet met de volgende gegevens per gemeente: bezoekadres, postadres, mailadres, telefoonnummer, website, plaatsen in de gemeente en aantal zetels in de gemeenteraad. Omdat al deze gegevens te vinden zijn op almanak.overheid.nl, gaan we de informatie geautomatiseerd downloaden van die website (scrapen) en samenvoegen in een CSV-bestand.

Hoe ga ik dat doen?
1. Verzamelen van alle links naar webpagina's met deze informatie voor een gemeente (dus scrapen links)
2. Scrapen van al die pagina's voor de benodigde info per gemeente

**1. Verzamelen links**  
Alle urls staan op de pagina https://almanak.overheid.nl/Gemeenten/
- alle links verzamelen van die pagina
- filteren zodat ik alleen gemeente-urls over houd
- gemeente urls opslaan in een lijstje

**2. Verzamelen informatie**  
Alle informatie per gemeente staat op een pagina zoals https://almanak.overheid.nl/Gemeenten/28617/Gemeente_Loppersum  
- maak een leeg csv-bestand
- begin een for-loop, voor elke url:
- vraag url op
- maak een soepje
- selecteer benodigde data
- breek post- en bezoekadres op
- verzamel alle info in een rij
- schrijf rij naar csv-bestand
- na for-loop sluit csv-bestand

## 1. verzamelen links

In [18]:
# vraag pagina met links naar alle gemeenten op
r = requests.get('https://almanak.overheid.nl/Gemeenten/')

In [19]:
# controleer of request gelukt is; succes = 200 response
r

<Response [200]>

In [20]:
# maak een soepje van de content van de pagina
soup = BeautifulSoup(r.content, 'html.parser')

In [21]:
# controleer of de soup is wat je verwacht
soup

<!DOCTYPE html>

<html lang="nl">
<head>
<title>Contactgegevens Gemeenten | Overheid.nl</title>
<link href="/static/onl/css/generic.css" rel="stylesheet" title="default"/>
<link href="/static/onl/images/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="https://almanak.overheid.nl/Gemeenten" name="DCTERMS.identifier" scheme="XSD.anyURI"/>
<meta content="Contactgegevens Gemeenten | Overheid.nl" name="DCTERMS.title"/>
<meta content="overzichtspagina" name="DCTERMS.type" scheme="OVERHEID.Informatietype"/>
<meta content="Het Register van Overheidsorganisaties bevat de namen, adressen en andere contactgegevens van de Nederlandse overheidsorganisaties. Ook de persoonsgegevens van (voornamelijk) leidinggevende personen binnen de Nederlandse overheid zijn opgenomen." name="DCTERMS.description"/>
<meta content="nl-NL" 

In [22]:
# haal alle links uit de soep met BeautifulSoup
soup.find_all('a')

[<a href="https://www.overheid.nl/"><img alt="Overheid.nl" src="/static/onl/images/logo.svg"/></a>,
 <a href="#content">Direct naar de inhoud</a>,
 <a data-decorator="init-toggle-other-sites" data-handler="toggle-other-sites" href="#other-sites"><span class="visually-hidden">Andere sites binnen </span>Overheid.nl</a>,
 <a class="hidden-desktop" data-decorator="init-toggle-other-sites" data-handler="toggle-other-sites" href="#other-sites"><span class="visually-hidden">Andere sites binnen </span>Overheid.nl</a>,
 <a href="https://www.overheid.nl/aankondigingen-over-uw-buurt">Naar aankondigingen over uw buurt</a>,
 <a href="https://www.overheid.nl/dienstverlening">Naar dienstverlening</a>,
 <a href="https://www.overheid.nl/beleid-en-regelgeving">Naar beleid &amp; regelgeving</a>,
 <a href="/">Naar overheidsorganisaties</a>,
 <a href="https://www.overheid.nl/">Home</a>,
 <a href="/">Contactgegevens overheden</a>,
 <a href="/Gemeenten">Gemeenten</a>,
 <a href="/22179/Gemeente_Aa_en_Hunze">G

Links naar gemeenten zien er zo uit:  
 '/28617/Gemeente_Loppersum',  
 '/25008/Gemeente_Echt-Susteren',  
 '/39441/Gemeente_Sint-Michielsgestel',  
 '/27951/Gemeente_Kaag_en_Braassem',  
 '/35518/Gemeente_Castricum',  
 '/26989/Gemeente_Oisterwijk',  
 '/33233/Gemeente_Bernheze',  
 '/31060/Gemeente_Son_en_Breugel',  
 '/24805/Gemeente_Nuenen_ca',  
 '/37098/Gemeente_Langedijk'  

Omdat we alleen links naar gemeenten willen, ga ik de lijst met alle links filteren.   
  
De eigenschap die alle links die ik wil hebben onderscheid van de rest is het volgende format:  
/*een aantal getallen*/Gemeente_*naam van de gemeente*  

De filter maak ik met een reguliere expressie. Je kunt de regex testen op [http://regex101.com](http://regex101.com).

Mijn regex-filter zoekt naar een / gevolgd door meer dan 1 getal gevolgd door *whatever*: ^\/[0-9]+.*
  
    
  ----
  
Om te testen of mijn filter werkt, match ik alle links uit de soep tegen de filter. Omdat er meer links zijn die ik wel wil, dan die ik niet wil; laat ik de computer alleen de links die NIET matchen printen.

In [25]:
# voor elke link in de soep vind...
for i in soup.find_all('a'):
    # kijk of de link matcht met de reguliere expressie
    if re.match(r'^\/[0-9]+.*', i['href']):
            # als dat het geval is, doe niets en ga door
            continue
    # als dat niet het geval is...
    else:
        # print de link
        print(i['href'])

https://www.overheid.nl/
#content
#other-sites
#other-sites
https://www.overheid.nl/aankondigingen-over-uw-buurt
https://www.overheid.nl/dienstverlening
https://www.overheid.nl/beleid-en-regelgeving
/
https://www.overheid.nl/
/
/Gemeenten
/
/Gemeenten/A-Z/A
/export/Gemeenten.csv
#
https://www.overheid.nl/over-deze-site
https://www.overheid.nl/contact
https://www.overheid.nl/english
https://www.overheid.nl/help
https://www.overheid.nl/help/zoeken
https://www.overheid.nl/informatie-hergebruiken
https://www.overheid.nl/privacy-statement
https://www.overheid.nl/toegankelijkheid
https://www.overheid.nl/sitemap
https://data.overheid.nl/
http://linkeddata.overheid.nl
https://puc.overheid.nl/
https://mijn.overheid.nl/
https://www.rijksoverheid.nl/
https://ondernemersplein.kvk.nl/
https://www.werkenbijdeoverheid.nl/


Dat werkte prima, dus ga ik nu alle links verzamelen. 
  
Let op: de filter moet ik dus nog wel even herschrijven. 
Ik maak een lege lijst aan met alle urls, ik noem de lijst `urls`.  
Verderop in de code voeg ik de gemeente url toe aan de lijst met de `.append()`-functie.

In [26]:
# maak lege lijst om urls in op te slaan
urls = []
# request om pagina met links naar gemeenten op te halen
r = requests.get('https://almanak.overheid.nl/Gemeenten/')
# als gelukt, ...
if r.status_code == 200:
    # maak een soepje
    soup = BeautifulSoup(r.content, 'html.parser')
    # voor elke link in alle links uit het soepje...
    for i in soup.find_all('a'):
        # ...kijk of de link begint met een / gevolgd door getallen (mbv regex)
        if re.match(r'^\/[0-9]+.*', i['href']):
            # als dat het geval is, voeg de link dan toe aan de urls-lijst
            urls.append(i['href'])
        # als dat niet het geval is...
        else:
            # doe dan niets, en ga verder met de for-loop
            pass

In [27]:
# controleer de lengte van de lijst urls
# omdat er in NL 355 gemeenten zijn, verwacht ik dat dat ook de lengte is van de lijst
len(urls)

355

In [29]:
# controleer de lijst door naar een steekproef van 10 te kijken
# als je deze code een paar keer draait neem je in een paar seconde zo een steekproef van 40 of 50
random.sample(urls, 10)

['/38312/Gemeente_Soest',
 '/21320/Gemeente_Peel_en_Maas',
 '/30535/Gemeente_Halderberge',
 '/39606/Gemeente_Valkenburg_aan_de_Geul',
 '/31991/Gemeente_Westerveld',
 '/31842/Gemeente_Hilvarenbeek',
 '/33912/Gemeente_Bodegraven-Reeuwijk',
 '/23981/Gemeente_Westervoort',
 '/30578/Gemeente_Oudewater',
 '/33032/Gemeente_Gemert-Bakel']

Ziet er ook prima uit, dus nu kunnen we de informatie gaan verzamelen per gemeente...

## 2. Verzamelen informatie

De to do lijst voor dit deel zag er zo uit:
- maak een leeg csv-bestand
- begin een for-loop, voor elke url:
    - vraag url op
    - maak een soepje
    - selecteer benodigde data
        - gemeentenaam
        - post- en bezoekadres op
        - telefoonnummer
        - website
        - aantal zetels
        - plaatsen in gemeenten
        - verzamel alle info in een rij
    - verzamel alle info in een rij
    - schrijf rij naar csv-bestand
- na for-loop sluit csv-bestand

Maar de volgorde van het schrijven van de code ziet er net anders uit. Net als in de cursus, ga ik de code voor het maken van een csv-bestand en de informatie daar naar toe schrijven als laatste toevoegen. 

Als eerste ga ik de naam van de gemeenten selecteren uit de url. Omdat ik hiervoor de pagina niet hoef op te vragen, doe ik dat eerst. 

Daarna vraag ik de pagina op, en selecteer de info uit het soepje. Dan verzamel ik alles in een rij. Daarna maak ik een csv-bestand zodat ik de informatie kan wegschrijven naar dat bestand.

In [31]:
# omdat dit een test is, gebruik ik alleen de eerste 5 rijen van de lijst met urls
urls[:5]

# om alleen de eerste 5 urls te selecteren 'slice' ik de lijst
# je gebruikt [:5] voor het selecteren van de eerste 5, 
# gebruik [5:] voor het selecteren van alles behalve de eerste 5,
# gebruik [350:] voor het selecteren van de laatste 5 (de lijst is 355 lang in totaal)
# gebruik [1:5] voor het selecteren van 1 tot 5, dus exclusief 5
# let op: computers beginnen met tellen bij 0!

['/22179/Gemeente_Aa_en_Hunze',
 '/38409/Gemeente_Aalsmeer',
 '/36483/Gemeente_Aalten',
 '/35934/Gemeente_Achtkarspelen',
 '/29079/Gemeente_Alblasserdam']

Ik wil de gemeentenaam uit de url halen, dus gebruik ik een regex om de naam te selecteren met behulp van de `re.search(regex, input)` functie. Ik gebruik `.group(1)` om de juiste groep uit mijn regex te selecteren. Omdat 'Aa_en_Hunze' niet is wat ik wil, vervang ik elke _ met een spatie met behulp van de `.replace()`-functie.

In [32]:
# voor elke rij in urls 0 tot 5:
for i in urls[:5]:
    # de variabele gemeente is de 1e groep uit de regex-match, waarbij alle _ vervangen worden door spaties
    gemeente = re.search(r'\/Gemeente_(.*)', i).group(1).replace('_', ' ')
    # om te testen of dit werkt, print ik de gemeentenamen
    print(gemeente)

Aa en Hunze
Aalsmeer
Aalten
Achtkarspelen
Alblasserdam


Ik ga voor het eerst een pagina op vragen. Omdat ik net handmatig heb gecontroleerd en bevestigd dat de pagina van Aa en Hunze een exemplarische pagina is, test ik mijn scraper alleen op die url. 

Voor het opvragen van de . url gebruik ik de `requests.get(url)` functie uit de requests library. Als het is lukt, geeft `r.status_code` mij een 200 response.

In [34]:
# dit is een test, dus doe dit alleen voor de eerste url
for i in urls[:1]:
    # vraag de site op
    r = requests.get('https://almanak.overheid.nl'+i)
    # controleer op het gelukt is
    print(r.status_code)

200


Maak met behulp van de Beautiful Soup library een soepje van de content van de opgevraagde webpagina. Dit kan met de `BeautifulSoup(r.content, parser)` functie uit de Beautiful Soup library.    

In [35]:
soup = BeautifulSoup(r.content, 'html.parser')

In [36]:
# controleer of dit werkte, print de soup
# als het goed is, bevat de soup de broncode van de pagina...
soup

<!DOCTYPE html>

<html lang="nl">
<head>
<title>Contactgegevens Gemeente Aa en Hunze | Overheid.nl</title>
<link href="/static/onl/css/generic.css" rel="stylesheet" title="default"/>
<link href="/static/onl/images/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="https://almanak.overheid.nl/22179/Gemeente_Aa_en_Hunze" name="DCTERMS.identifier" scheme="XSD.anyURI"/>
<meta content="Contactgegevens Gemeente Aa en Hunze | Overheid.nl" name="DCTERMS.title"/>
<meta content="overzichtspagina" name="DCTERMS.type" scheme="OVERHEID.Informatietype"/>
<meta content="Het Register van Overheidsorganisaties bevat de namen, adressen en andere contactgegevens van de Nederlandse overheidsorganisaties. Ook de persoonsgegevens van (voornamelijk) leidinggevende personen binnen de Nederlandse overheid zijn opgenomen." name="DCTERM

Selecteer de tabel uit de soep. Gebruik de `.find_all(element)` formule om alle tabellen te vinden. Gebruik `[0]` om aan te geven welke tabel je precies nodig hebt.

In [37]:
tabel1 = soup.find_all('table')[0]

In [38]:
# print tabel1 om te zien of het werkte
tabel1

<table class="table__data-overview" data-roo-element="element-adresgegevens">
<tr>
<th scope="row">Bezoekadres</th>
<td data-before="Bezoekadres">
					Spiekersteeg 1<br/>
					9461 BH GIETEN
				</td>
</tr>
<tr>
<th scope="row">Postadres</th>
<td data-before="Postadres">
					
					Postbus 93<br/>
					9460 AB GIETEN
				</td>
</tr>
<tr>
<th scope="row">Telefoon</th>
<td data-before="Telefoon">
						14 0592 (algemeen)
						
						
						
					</td>
</tr>
<tr>
<th scope="row">Internet</th>
<td data-before="Internet">
<a href="http://www.aaenhunze.nl">http://www.aaenhunze.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
<tr>
<th scope="row">E-mail</th>
<td data-before="E-mail">
<a href="mailto:gemeente@aaenhunze.nl">gemeente@aaenhunze.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
</table>

Een overzicht van alle informatie die we uit deze tabel1 willen halen:
- bezoekadres; later opsplitsen in adres, postcode, plaats
- postadres; later opsplitsen in adres, postcode, plaats
- telefoon
- e-mail
- website

Voor het selecteren van de informatie die we willen, gebruiken we `.find_all('td')` in combinatie met `[0]` of `[1]` etc. (afhankelijk van wat je nodig hebt. Met `.get_text()` selecteer je alleen de tekst uit een element; met `.strip()` verwijder je spaties rondom die tekst; met `.replace(te vervangen, vervanging)` vervang je onderdelen uit de tekst.

In [41]:
# zoek in de eerste tabel naar alle td's
tabel1.find_all('td')

[<td data-before="Bezoekadres">
 					Spiekersteeg 1<br/>
 					9461 BH GIETEN
 				</td>, <td data-before="Postadres">
 					
 					Postbus 93<br/>
 					9460 AB GIETEN
 				</td>, <td data-before="Telefoon">
 						14 0592 (algemeen)
 						
 						
 						
 					</td>, <td data-before="Internet">
 <a href="http://www.aaenhunze.nl">http://www.aaenhunze.nl</a>
 						 (algemeen)
 						
 						
 					</td>, <td data-before="E-mail">
 <a href="mailto:gemeente@aaenhunze.nl">gemeente@aaenhunze.nl</a>
 						 (algemeen)
 						
 						
 					</td>]

In [43]:
# het bezoekadres staat in de 1e td
bezoekadres = tabel1.find_all('td')[0]

In [44]:
# controleer het bezoekadres
bezoekadres

<td data-before="Bezoekadres">
					Spiekersteeg 1<br/>
					9461 BH GIETEN
				</td>

In [45]:
# dat is meer dan we willen, geef alleen de tekst computer...
bezoekadres = tabel1.find_all('td')[0].get_text()

In [46]:
# controleer het bezoekadres
bezoekadres

'\n\t\t\t\t\tSpiekersteeg 1\n\t\t\t\t\t9461 BH GIETEN\n\t\t\t\t'

In [47]:
# strip de tekst van onzin rondom het bezoekadres zelf, computer...
bezoekadres = tabel1.find_all('td')[0].get_text().strip()

In [48]:
# controleer het bezoekadres
bezoekadres

'Spiekersteeg 1\n\t\t\t\t\t9461 BH GIETEN'

In [49]:
# beter, maar nog niet helemaal... laten we alle \t vervangen met niets
bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','')

In [50]:
# controleer het bezoekadres
bezoekadres

'Spiekersteeg 1\n9461 BH GIETEN'

In [51]:
# als we de \n vervangen met een spatie, dan zijn we er wel...
bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','').replace('\n', ' ')

In [52]:
# controleer het bezoekadres
bezoekadres

'Spiekersteeg 1 9461 BH GIETEN'

Ik wil het bezoekadres opsplitsen in straat + nummer; postcode; en plaats. Dat ga ik doen met behulp van `re.search(regex, input)` en `.group()`. Ik verdeel het bezoekadres (een groep in een regex staat tussen`(` en `)`) in 3 groepen (straat + nummer; postcode; plaats), en gebruik `.group()` om steeds 1 van die groepen te selecteren. 

De reguliere expressie die ik hiervoor gebruik ziet er zo uit: *(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)*

Je kunt dit testen door het bezoekadres 'Spiekersteeg 1 9461 BH GIETEN' in te vullen op [https://regex101.com](https://regex101.com).

Het adres deel van het bezoekadres ga ik opslaan onder variabele baAdres (bezoek adres adres).

In [54]:
# gebruik regex search om deze regex te vinden in het bezoekadres; geef daarna alleen de 1e groep terug.
re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)

'Spiekersteeg 1'

In [55]:
# deze info opslaan naar variabele baAdres
baAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)

In [57]:
# check
baAdres

'Spiekersteeg 1'

Vooraf wist ik al dat ik niet alleen het adres van het bezoekadres apart wilde hebben, maar ook de postcode en de plaats. Daarom had ik in mijn regex direct 3 groepen gemaakt, *(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)*. 

Nu kan ik mijn regex-code dus hergebruiken, ik moet alleen het groep-nummer aanpassen.

In [58]:
# gebruik re.search om het bezoekadres te doorzoeken; selecteer de tweede groep
re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)

'9461 BH'

In [59]:
# postcode opslaan naar variabele
baPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)

In [60]:
# check
baPostcode

'9461 BH'

In [61]:
# gebruik re.search om het bezoekadres te doorzoeken; selecteer de derde groep
re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)

'GIETEN'

In [62]:
# postcode opslaan naar variabele
baPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)

In [63]:
# check
baPlaats

'GIETEN'

Vervolgens doe ik precies hetzelfde voor het postadres: in tabel1 zoek ik naar naar de tweede *table data* (`.find_all('td')[1]`); en daarna splits ik het postadres op in paAdres, paPostcode en paPlaats (`re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres)`). 

Omdat ik wil dat de computer ook doorgaat als er iets niet lukt gebruik ik `try` en `except`. Daarmee zeg ik tegen de computer 'probeer dit te doen (opdracht na `try`), maar mocht dat nou niet lukken doe dan dit (alternatieve opdracht na `except`)'. 

Goed om te weten: `None` is computerjargon voor 'niets' of een lege cel. Door `None` te gebruiken weet ik zeker dat de computer het wel geprobeerd heeft en mijn programmaatje wel werkte, maar dat het om een of andere reden toch niet gelukt is.

In [68]:
# probeer het postadres uit tabel 1 te halen:
# navigeer naar 2e td, haal tekst eruit, strip spaties etc., vervang \t en \n
try:
    postadres = tabel1.find_all('td')[1].get_text().strip().replace('\t','').replace('\n', ' ')
except:
    postadres = None
# opsplitsen van postadres in paAdres, paPostcode en paPlaats
try:
    paAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(1)
except:
    paAdres = None
try:
    paPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(2)
except:
    paPostcode = None
try:
    paPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(3)
except:
    paPlaats = None

In [73]:
# te controle print ik de variablen...
print(postadres)
print(paAdres)
print(paPostcode)
print(paPlaats)

Postbus 93 9460 AB GIETEN
Postbus 93
9460 AB
GIETEN


Verder wil ik uit tabel1 nog het telefoonnummer en de site halen...

In [76]:
# probeer td 3 te vinden met het telefoonnummer in
try:
    # ik strip het telefoonnummer meteen van alle niet haken en (algemeen) etc...
    telefoon = tabel1.find_all('td')[2].get_text().strip().replace(' (algemeen)', '').replace('(', '').replace(')','')
except:
    telefoon = None

In [77]:
# controle
print(telefoon)

14 0592


Het mailadres kun je op verschillende manieren uit de tabel halen. 

De eerste manier lijkt op de methode waarmee we aan het postadres zijn gekomen:
- haal de table data (td) op
- strip van alle gedoe er om heen
- vervang alle \t en \n en (algemeen) etc.

In [94]:
# haal het mailadres uit td 5
mailadres = tabel1.find_all('td')[4].get_text().strip().replace('\n','').replace('\t','').replace(' (algemeen)', '')
mailadres

'gemeente@aaenhunze.nl'

De tweede manier om aan het mailadres te komen vind ik iets mooier. Maar over smaak valt te twisten. 

Als je de td ophaalt, zie je namelijk dat het mailadres ook als een href bij de link staat... 
(Let op: als er mailadressen zijn die niet 'klikbaar' zijn, omdat er geen link van is gemaakt dan werkt deze methode niet.)

In [95]:
# even naar de td kijken...
tabel1.find_all('td')[4]

<td data-before="E-mail">
<a href="mailto:gemeente@aaenhunze.nl">gemeente@aaenhunze.nl</a>
						 (algemeen)
						
						
					</td>

In [96]:
# vanaf hier kun je naar de link navigeren...
tabel1.find_all('td')[4].find('a')

<a href="mailto:gemeente@aaenhunze.nl">gemeente@aaenhunze.nl</a>

In [97]:
# selecteer de href
tabel1.find_all('td')[4].find('a')['href']

'mailto:gemeente@aaenhunze.nl'

In [98]:
# vervang 'mailto:'
tabel1.find_all('td')[4].find('a')['href'].replace('mailto:', '')

'gemeente@aaenhunze.nl'

In [99]:
# alternatieve manier om het mailadres uit td 5 te halen
mail = tabel1.find_all('td')[4].find('a')['href'].replace('mailto:', '')
mail

'gemeente@aaenhunze.nl'

In [101]:
# natuurlijk is .get_text() sneller, maar als de werkelijlk link verschilt 
# van de tekst dan heb je alleen de tekst en niet de werkelijke link-bestemming zelf...
mail2 = tabel1.find_all('td')[4].find('a').get_text()
mail2

'gemeente@aaenhunze.nl'

Ook de website kun je op meerdere manieren uit de tabel halen...

Het verschil zit 'm in precies welke informatie je selecteert; ook al is die informatie nu precies hetzelfde...
Ok, komtie. Dit is de inhoud van de 4e cel van de tabel:

In [112]:
tabel1.find_all('td')[3]

<td data-before="Internet">
<a href="http://www.aaenhunze.nl">http://www.aaenhunze.nl</a>
						 (algemeen)
						
						
					</td>

<img src="websiteUitleg.png">

Met de eerste methode haal je de gele url uit de tabel; met de tweede methode haal je de paarse url uit de tabel. 

In het geel zie je de link waarnaar je wordt verwezen als je klikt; in het paars zie je de link zoals die wordt getoond aan website bezoekers. In dit geval is dat dezelfde informatie, maar dat is niet altijd zo. En in mijn ervaring wil je dan vaker liever de link waarnaar je wordt verwezen, dan de site die wordt getoond.

In [107]:
# haal de website op - methode 1
tabel1.find_all('td')[3].find('a')['href']

'http://www.aaenhunze.nl'

In [113]:
# haal de website op - methode 2
tabel1.find_all('td')[3].get_text().strip().replace('\n','').replace('\t','').replace(' (algemeen)', '')

'http://www.aaenhunze.nl'

Uit tabel2, die we nog moeten maken, wil ik nog de plaatsen en aantal zetels ophalen... Here we go:

In [114]:
tabel2 = soup.find_all('table')[1]

In [115]:
# controle
tabel2

<table class="table__data-overview" data-roo-element="element-basisgegevens">
<tr>
<th scope="row">
										Organisatietype
									</th>
<td data-before="Organisatietype">
										
											
												
													
		Gemeente
	
												
												
											
										
									</td>
</tr>
<tr>
<th scope="row">Oppervlakte</th>
<td data-before="Oppervlakte">279.27 km<sup>2</sup></td>
</tr>
<tr>
<th scope="row">Aantal inwoners</th>
<td data-before="Aantal inwoners">25203</td>
</tr>
<tr>
<th scope="row">Inwoners per km<sup>2</sup></th>
<td data-before="Inwoners per km2">90</td>
</tr>
<tr>
<th scope="row">Plaatsen binnen deze gemeente</th>
<td data-before="Plaatsen binnen deze gemeente">Amen, Anderen, Anloo, Annen, Annerveenschekanaal, Balloerveld, Balloo, Deurze, Eext, Eexterveen, Eexterveenschekanaal, Eexterzandvoort, Ekehaar, Eldersloo, Eleveld, Gasselte, Gasselternijveen, Gasselternijveenschemond, Gasteren, Geelbroek, Gieten, Gieterveen, Grolloo, Marwijksoord, Nieuw A

In [116]:
# tabel2 td5
tabel2.find_all('td')[4]

<td data-before="Plaatsen binnen deze gemeente">Amen, Anderen, Anloo, Annen, Annerveenschekanaal, Balloerveld, Balloo, Deurze, Eext, Eexterveen, Eexterveenschekanaal, Eexterzandvoort, Ekehaar, Eldersloo, Eleveld, Gasselte, Gasselternijveen, Gasselternijveenschemond, Gasteren, Geelbroek, Gieten, Gieterveen, Grolloo, Marwijksoord, Nieuw Annerveen, Nieuwediep, Nijlande, Nooitgedacht, Oud Annerveen, Papenvoort, Rolde, Schipborg, Schoonloo, Spijkerboor DR, Vredenheim</td>

In [117]:
try:
    plaatsen = tabel2.find_all('td')[4].get_text().strip()
except:
    plaatsen = None

In [118]:
# check
plaatsen

'Amen, Anderen, Anloo, Annen, Annerveenschekanaal, Balloerveld, Balloo, Deurze, Eext, Eexterveen, Eexterveenschekanaal, Eexterzandvoort, Ekehaar, Eldersloo, Eleveld, Gasselte, Gasselternijveen, Gasselternijveenschemond, Gasteren, Geelbroek, Gieten, Gieterveen, Grolloo, Marwijksoord, Nieuw Annerveen, Nieuwediep, Nijlande, Nooitgedacht, Oud Annerveen, Papenvoort, Rolde, Schipborg, Schoonloo, Spijkerboor DR, Vredenheim'

In [120]:
# tabel2 td6
tabel2.find_all('td')[5]

<td data-before="Totaal aantal zetels gemeenteraad">21</td>

In [121]:
try:
    zetels = tabel2.find_all('td')[5].get_text().strip()
except: 
    zetels = None

In [122]:
# check
zetels

'21'

Wat nog rest is de code om een csv-bestand mee te maken en een `writer` te maken waarmee je data naar die csv kunt schrijven. 

Alle onderdelen gecombineerd, levert dat de volgende scraper op:

## Scraper

In [123]:
# create a csv file to save data in
f = open('190721 contactgegevens gemeenten.csv', 'w', encoding='utf8', newline='')
# create a writer to write data with to the newly created csv-file
writer = csv.writer(f, delimiter=',')
# use the writer to write the first row, the column headers, to the file
writer.writerow(['gemeente', 'baAdres', 'baPostcode', 'baPlaats', 'paAdres', 'paPostcode', 
                 'paPlaats', 'mailadres', 'website', 'telefoon', 'plaatsen', 'zetels'])
# voor elke url uit de lijst... (nu even beperkt tot de 1e tien, want alleen maar testen)
for i in urls:
    # vraag de site op
    r = requests.get('https://almanak.overheid.nl'+i)
    # als gelukt...
    if r.status_code == 200:
        # maak een soepje
        soup = BeautifulSoup(r.content, 'html.parser')
        # selecteer de tabellen
        tabel1 = soup.find_all('table')[0]
        tabel2 = soup.find_all('table')[1]
        # haal uit tabel1: bezoekadres, postadres, telefoon, site, email
        # probeer het bezoekadres uit tabel 1 te halen:
        # navigeer naar 1e td, haal tekst eruit, strip spaties etc., vervang \t en \n
        try:
            bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','').replace('\n', ' ')
        # als dat niet lukt, dan is bezoekadres None
        except:
            bezoekadres = None
        # opsplitsen van bezoekadres in baAdres, baPostcode en baPlaats
        try:
            baAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)
        except:
            baAdres = None
        try:
            baPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)
        except:
            baPostcode = None
        try:
            baPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)
        except:
            baPlaats = None
        # probeer het postadres uit tabel 1 te halen:
        # navigeer naar 2e td, haal tekst eruit, strip spaties etc., vervang \t en \n
        try:
            postadres = tabel1.find_all('td')[1].get_text().strip().replace('\t','').replace('\n', ' ')
        except:
            postadres = None
        # opsplitsen van postadres in paAdres, paPostcode en paPlaats
        try:
            paAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(1)
        except:
            paAdres = None
        try:
            paPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(2)
        except:
            paPostcode = None
        try:
            paPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(3)
        except:
            paPlaats = None
        # haal telefoonnummer, mailadres, en website op
        try:
            mailadres = tabel1.find_all('td')[4].find('a')['href'].replace('mailto:', '')
        except:
            mailadres = None
        try:
            website = tabel1.find_all('td')[3].find('a')['href']
        except:
            website = None
        try:
            telefoon = tabel1.find_all('td')[2].get_text().strip().replace(' (algemeen)', '').replace('(', '').replace(')','')
        except:
            telefoon = None
         # haal uit tabel2: plaatsen en zetels
        try:
            plaatsen = tabel2.find_all('td')[4].get_text().strip()
        except:
            plaatsen = None
        try:
            zetels = tabel2.find_all('td')[5].get_text().strip()
        except: 
            zetels = None
        try:
            gemeente = re.search(r'\/Gemeente_(.*)', i).group(1).replace('_', ' ')
        except:
            gemeente = None
        # verzamel alle datapunten in een lijst die 'data' heet
        data = [gemeente, baAdres, baPostcode, baPlaats, paAdres, paPostcode, paPlaats, 
                mailadres, website, telefoon, plaatsen, zetels]
        # schrijf de lijst 'data' als een rij naar de csv
        writer.writerow(data)
f.close()

## Controle

In [125]:
# laad de zojuist gemaakte csv in 
df = pd.read_csv('190721 contactgegevens gemeenten.csv')

In [126]:
# controleer de voorm van de csv; we verwachten natuurlijk 355 rijen en 12 kolommen
df.shape

(355, 12)

In [128]:
# bekijk een sample van 10 rijen van de dataframe 
df.sample(10)

,gemeente,baAdres,baPostcode,baPlaats,paAdres,paPostcode,paPlaats,mailadres,website,telefoon,plaatsen,zetels
114,Haaksbergen,Markt 3,7481 HS,HAAKSBERGEN,Postbus 102,7480 AC,HAAKSBERGEN,gemeente@haaksbergen.nl,http://www.haaksbergen.nl,053 573 45 67,Haaksbergen,18.0
246,Rhenen,Nieuwe Veenendaalseweg 75,3911 MG,RHENEN,Postbus 201,3910 AE,RHENEN,info@rhenen.nl,http://www.rhenen.nl,0317 68 16 81,"Elst UT, Rhenen",16.0
141,Hillegom,Hoofdstraat 115,2181 EC,HILLEGOM,Postbus 32,2180 AA,HILLEGOM,info@hillegom.nl,http://www.hillegom.nl,14 0252,Hillegom,19.0
0,Aa en Hunze,Spiekersteeg 1,9461 BH,GIETEN,Postbus 93,9460 AB,GIETEN,gemeente@aaenhunze.nl,http://www.aaenhunze.nl,14 0592,"Amen, Anderen, Anloo, Annen, Annerveenschekana...",21.0
162,Laarbeek,Koppelstraat 37,5741 GA,BEEK EN DONK,Postbus 190,5740 AD,BEEK EN DONK,gemeente@laarbeek.nl,http://www.laarbeek.nl,0492 46 97 00,"Aarle-Rixtel, Beek en Donk, Lieshout, Mariahout",19.0
4,Alblasserdam,Cortgene 2,2951 ED,ALBLASSERDAM,Postbus 2,2950 AA,ALBLASSERDAM,bestuurssecretariaat@alblasserdam.nl,http://www.alblasserdam.nl,14 078,Alblasserdam,19.0
88,Ede,Bergstraat 4,6711 DD,EDE GLD,Postbus 9022,6710 HK,EDE GLD,info@ede.nl,http://www.ede.nl,140318,"Bennekom, De Klomp, Deelen, Ede GLD, Ederveen,...",39.0
86,Echt-Susteren,Nieuwe Markt 55,6101 CV,ECHT,Postbus 450,6100 AL,ECHT,info@echt-susteren.nl,http://www.echt-susteren.nl,0475 47 84 78,"Echt, Koningsbosch, Maria Hoop, Nieuwstadt, Ro...",23.0
124,Heemskerk,Maerten v Heemskerckpln 1,1964 EZ,HEEMSKERK,Postbus 1,1960 AA,HEEMSKERK,MailContent@heemskerk.nl,http://www.heemskerk.nl,14 0251,Heemskerk,25.0
257,Scherpenzeel,Stationsweg 389 a,3925 CC,SCHERPENZEEL GLD,Postbus 100,3925 ZJ,SCHERPENZEEL GLD,secretariaat@scherpenzeel.nl,http://www.scherpenzeel.nl,033 277 23 24,Scherpenzeel GLD,13.0
